In [1]:
# タイタニック生存予測

In [2]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/kaggle/titanic

/content/drive/MyDrive/kaggle/titanic


In [4]:
import pandas as pd
import numpy as np

In [5]:
# データの読み込み
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [6]:
# データの先頭の５行を表示
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
# データの型を調べる
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [8]:
# nullかどうかを調べてから、その数を調べる
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
test.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [10]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [11]:
x = train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = train[['Survived']]

In [12]:
# one-hotエンコーディングをする
# 展開してそれぞれ0か１を入れる
x = pd.get_dummies(x, columns=['Pclass', 'Sex','Embarked'])
x.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1
1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0
2,26.0,0,0,7.9250,0,0,1,1,0,0,0,1
3,35.0,1,0,53.1000,1,0,0,1,0,0,0,1
4,35.0,0,0,8.0500,0,0,1,0,1,0,0,1


In [13]:
# nullの数を調べる
x['Age'].isnull().sum()

177

In [14]:
# nullを平均で埋める
x['Age'] = x['Age'].fillna(x['Age'].mean())
# 埋まったか確認
x['Age'].isnull().sum()

0

In [15]:
# 訓練データとテストデータに分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [16]:
# knnのインポート
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [17]:
knn.fit(x_train, y_train)
knn.score(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.8158682634730539

In [18]:
knn.score(x_test, y_test)

0.7354260089686099

In [19]:
# 標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [20]:
knn.fit(x_train_scaled, y_train)
knn.score(x_train_scaled, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.8667664670658682

In [21]:
knn.fit(x_test_scaled, y_test)


/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [22]:
# submit用に作る
x_for_submit = test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
submit = test[['PassengerId']]
x_for_submit['Age']  = x_for_submit['Age'].fillna(x_for_submit['Age'].mean())
x_for_submit['Fare'] = x_for_submit['Fare'].fillna(x_for_submit['Fare'].mean())
x_for_submit.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [23]:
x_for_submit = pd.get_dummies(x_for_submit, columns=['Pclass', 'Sex','Embarked'])
x_for_submit

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,34.50000,0,0,7.8292,0,0,1,0,1,0,1,0
1,47.00000,1,0,7.0000,0,0,1,1,0,0,0,1
2,62.00000,0,0,9.6875,0,1,0,0,1,0,1,0
3,27.00000,0,0,8.6625,0,0,1,0,1,0,0,1
4,22.00000,1,1,12.2875,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,30.27259,0,0,8.0500,0,0,1,0,1,0,0,1
414,39.00000,0,0,108.9000,1,0,0,1,0,1,0,0
415,38.50000,0,0,7.2500,0,0,1,0,1,0,0,1
416,30.27259,0,0,8.0500,0,0,1,0,1,0,0,1


In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_for_submit)
x_for_submit = scaler.transform(x_for_submit)
x_for_submit

array([[ 0.3349926 , -0.49947002, -0.4002477 , ..., -0.56814154,
         2.84375747, -1.35067551],
       [ 1.32553003,  0.61699237, -0.4002477 , ..., -0.56814154,
        -0.35164743,  0.74037028],
       [ 2.51417495, -0.49947002, -0.4002477 , ..., -0.56814154,
         2.84375747, -1.35067551],
       ...,
       [ 0.65196458, -0.49947002, -0.4002477 , ..., -0.56814154,
        -0.35164743,  0.74037028],
       [ 0.        , -0.49947002, -0.4002477 , ..., -0.56814154,
        -0.35164743,  0.74037028],
       [ 0.        ,  0.61699237,  0.61989583, ...,  1.76012477,
        -0.35164743, -1.35067551]])

In [25]:
submit['Survived'] = knn.predict(x_for_submit)
submit.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [26]:
# csvファイルを作成
submit.to_csv('submission/submit03.csv', index=False)